In [ ]:
import os
import time
import cv2

#os.system("v4l2-ctl --list-devices")
# set format to NTSC, thought PAL is higher resolution.  need to set cameras to PAL
for i in range(0, 4):
    cmd = f"v4l2-ctl --device /dev/video{i} --set-standard=ntsc"
    os.system(cmd)

cap = [None, None, None]
for i in range(0,3):
    # Open video capture object
    cap[i] = cv2.VideoCapture(i)

    # Check if video capture object is successfully opened
    if not cap[i].isOpened():
        print("Failed to open video capture object")
        exit()

    #set resolution to 640x480
    cap[i].set(3, 720)
    cap[i].set(4, 480)



In [ ]:
import matplotlib.pyplot as plt
frame = [None, None, None]
cap = [None, None, None]
for i in range(0,3):
    # Open video capture object
    cap[i] = cv2.VideoCapture(i)

    # Check if video capture object is successfully opened
    if not cap[i].isOpened():
        print("Failed to open video capture object")
        exit()

    #set resolution to 640x480
    cap[i].set(3, 720)
    cap[i].set(4, 480)

for i in range(0,3):

    ret, frame[i] = cap[i].read()

    # Check if frame is successfully read
    if not ret:
        print("Failed to read frame from video source")
        exit()


#create side by side image of i frames
frame_combined = cv2.hconcat([frame[0], frame[1], frame[2]])

# save image to disk
ts = int(time.time())
filename = f"frame_{ts}.jpg"
cv2.imwrite(filename, frame_combined)
#print(f"Frame saved to {filename} size:{frame_combined.shape}")


# Display the image
plt.figure(figsize=(20, 10))
plt.imshow(cv2.cvtColor(frame_combined, cv2.COLOR_BGR2RGB))
plt.show()
# Release the video capture object and close any open windows
for i in range(0,3):
    cap[i].release()


In [ ]:
import cv2
import pupil_apriltags
path = "/home/casaos/TrailerCamera/"

def point(pt):
    return (int(pt[0]),int(pt[1]))

def draw_corners(img, detection, color=(255)):
    pt1 = point(detection.corners[0])
    cv2.circle(img, pt1,5,color,-1)
    pt2 = point(detection.corners[1])
    cv2.line(img, pt1, pt2,color,2)
    pt3 = point(detection.corners[3])
    cv2.line(img, pt1, pt3,color,2)
    c = point(detection.center)
    cv2.circle(img, c,2,color,-1)
    cv2.putText(img,str(detection.tag_id), c, cv2.FONT_HERSHEY_PLAIN, 1.5,color,2)


w = int(img.shape[1]/3)
img_left = img[:,0:w]

#mirror image
img_left = cv2.flip(img_left, 1)

detector = pupil_apriltags.Detector()

result = detector.detect(img_left)

for i in range(len(result)):
   det = result[i]
   draw_corners(img_left, det)

# show grayscale iamge
plt.figure(figsize=(10, 10))  
plt.imshow(img_left, cmap='gray')


In [ ]:
import pupil_apriltags
import cv2
import os
import numpy as np
import json
import time
from AprilBoard import AprilBoard


def crop_image(image, bbox_str):
    """
    Crops an image based on a bounding box string format '[top:bottom,left:right]'.

    Args:
        image: A NumPy array representing the image.
        bbox_str: A string representing the bounding box in format '[top:bottom,left:right]'.

    Returns:
        A NumPy array representing the cropped image.
    """
    try:
        slices = bbox_str.strip('[]').split(',')
        top, bottom = map(int, slices[0].split(':'))
        left, right = map(int, slices[1].split(':'))
    except ValueError:
        raise ValueError('Invalid bounding box format. Use [top:bottom,left:right]')

    # Crop the image
    return image[top:bottom, left:right]


def point(pt):
    return (int(pt[0]),int(pt[1]))

def draw_detection(img, detection, color=(255,0,0)):
    pt1 = point(detection.corners[0])
    cv2.circle(img, pt1,5,color,-1)
    pt2 = point(detection.corners[1])
    cv2.line(img, pt1, pt2,color,2)
    pt3 = point(detection.corners[3])
    cv2.line(img, pt1, pt3,color,2)
    c = point(detection.center)
    cv2.circle(img, c,2,color,-1)
    cv2.putText(img,str(detection.tag_id), c, cv2.FONT_HERSHEY_PLAIN, 1.5,color,2)


In [ ]:
names = ["left", "back", "right"]
dir = '.'
allfiles = os.listdir(dir)
images = []
calibration_data = {}
for f in allfiles:
    if "unwarped" in f:
        continue
    if ".jpg" in f:
        images.append(os.path.join(dir,f))

    if 'intrinsics' in f:
        for n in names:
            if n in f:
                with open(f,'r') as fp:
                    calibration_data[n] = {'intrinsics':json.load(fp)}

board = AprilBoard.AprilBoard()
detector = pupil_apriltags.Detector()
images_unwarped = []

for path in images:
    print(path)
    img_raw = cv2.imread(path)

    views = []
    for i in range(len(names)):
        n = names[i]

        K = np.array(calibration_data[n]['intrinsics']['matrix'])
        D = np.array(calibration_data[n]['intrinsics']['distortion'])
        fisheyeMode = calibration_data[n]['intrinsics']['fisheye']
        flip = calibration_data[n]['intrinsics']['flip']
        crop = calibration_data[n]['intrinsics']['crop']
        img = crop_image(img_raw, crop)
        if flip:
            img = cv2.flip(img, 1)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        result = detector.detect(gray)

        for detection in result:
            draw_detection(img, detection,(0,255,0))
        if 'map1' not in calibration_data[name]:
            if fisheyeMode:
                calibration_data[name]['map1'], calibration_data[name]['map2'] = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, (img.shape[1], img.shape[0]), cv2.CV_16SC2)
            else:
                calibration_data[name]['map1'], calibration_data[name]['map2'] = cv2.initUndistortRectifyMap(K, D, np.eye(3), K, (img.shape[1], img.shape[0]), cv2.CV_16SC2)
        img2 = cv2.remap(img, calibration_data[name]['map1'], calibration_data[name]['map2'], interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
        views.append(img2)

    img_concat = cv2.hconcat(views)
    images_unwarped.append(img_concat)


#show grid of images in unwarped 4 columns no margin
cols = 2
rows = int(len(images_unwarped)/cols)
fig, ax = plt.subplots(rows, cols, figsize=(cols*6*3, rows*4))

# Remove space between subplots
plt.subplots_adjust(wspace=0, hspace=0)

for i in range(rows):
    for j in range(cols):
        ax[i,j].imshow(cv2.cvtColor(images_unwarped[i*cols+j], cv2.COLOR_BGR2RGB))
        ax[i,j].axis('off')

plt.show()

